In [2]:
import pandas as pd

url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

1. table is identified by html class attribute, first row is for header.

In [3]:
df=pd.read_html(url, attrs={'class': 'sortable'}, header=0);

2. read_html returns a list of dataframes. in this case i want the only that class=sortable
The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood

In [4]:
myWorkingDF = df[0]

3. filer data that borought is different tha 'Not assigned'
Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [5]:
myWorkingDF = myWorkingDF[myWorkingDF.Borough != 'Not assigned']

4. group by postcode e borougt to join multiple rows in Neighbourhood coloumn separated by ','
More than one neighborhood can exist in one postal code area.

In [6]:
myWorkingDF = myWorkingDF.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(','.join).reset_index()

5. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [7]:
myWorkingDF.loc[myWorkingDF['Neighbourhood']=='Not assigned', 'Neighbourhood']=myWorkingDF['Borough']
myWorkingDF.shape

(103, 3)

6. add lat & long to my dataframe, joined by postcode column; prints 16 firsts rows

In [8]:
geodata = pd.read_csv('http://cocl.us/Geospatial_data')
geodata = geodata.rename(columns={"Postal Code": "Postcode"})
geodata
myNewDF = pd.merge(myWorkingDF, geodata, on='Postcode')
myNewDF.head(16)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge,Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood,Morningside,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park,Ionview,Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea,Golden Mile,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest,Cliffside,Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff,Cliffside West",43.692657,-79.264848


In [96]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

address = 'Scarborough'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Scarborough are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Scarborough are 54.2847601, -0.4090339.


In [100]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, borough, neighborhood in zip(myNewDF['Latitude'], myNewDF['Longitude'], myNewDF['Borough'], myNewDF['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork